In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/bi-encoder-training-reformulated-data-v1/__results__.html
/kaggle/input/bi-encoder-training-reformulated-data-v1/__notebook__.ipynb
/kaggle/input/bi-encoder-training-reformulated-data-v1/__output__.json
/kaggle/input/bi-encoder-training-reformulated-data-v1/custom.css
/kaggle/input/bi-encoder-training-reformulated-data-v1/CLEAR/model/bi-encoder_distilroberta-base-2024-01-06_00-41-41/config.json
/kaggle/input/bi-encoder-training-reformulated-data-v1/CLEAR/model/bi-encoder_distilroberta-base-2024-01-06_00-41-41/merges.txt
/kaggle/input/bi-encoder-training-reformulated-data-v1/CLEAR/model/bi-encoder_distilroberta-base-2024-01-06_00-41-41/README.md
/kaggle/input/bi-encoder-training-reformulated-data-v1/CLEAR/model/bi-encoder_distilroberta-base-2024-01-06_00-41-41/tokenizer.json
/kaggle/input/bi-encoder-training-reformulated-data-v1/CLEAR/model/bi-encoder_distilroberta-base-2024-01-06_00-41-41/vocab.json
/kaggle/input/bi-encoder-training-reformulated-data-v1/CLEAR/model/bi-enc

In [2]:
def read_json_with_frozenset_column(path, frozenset_column_name):
  df = pd.read_json(path, orient='records')
  df[frozenset_column_name] = df[frozenset_column_name].apply(lambda x: frozenset(x))
  return df

In [3]:
!pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 6.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - \ done
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=c6d7e476e3e8e6a5caa54c677ad8a04d87a24a828b8679c6c69e0376f96d9248
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [4]:
import json
from sentence_transformers import SentenceTransformer, CrossEncoder, util
import time
import gzip
import os
import torch
import pandas as pd

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [5]:
if not torch.cuda.is_available():
    print("Warning: No GPU found. Please add GPU to your notebook")
else:
    print("OK")

OK


In [6]:
model_name = '/kaggle/input/bi-encoder-training-reformulated-data-v1/CLEAR/model/bi-encoder_distilroberta-base-2024-01-06_00-41-41'
bi_encoder = SentenceTransformer(model_name)
top_k = 100

cross_encoder = CrossEncoder('/kaggle/input/cross-encoder-training-reformulated-data-v1/CLEAR/model/cross-encoder-30_iter_TinyBERT-2024-01-05_23-44-23')

In [7]:
data_folder = '/kaggle/input/clear-subset'

In [8]:
test_corpus_subset = []

collection_filepath = os.path.join(data_folder, 'str_test_corpus_subset.json')
with open(collection_filepath, 'r', encoding='utf8') as fIn:
  data = json.load(fIn)
test_corpus_subset = data

print(type(test_corpus_subset))

<class 'list'>


In [9]:
test_answers_subset = []

collection_filepath = os.path.join(data_folder, 'str_test_answers_subset.json')
with open(collection_filepath, 'r', encoding='utf8') as fIn:
  data = json.load(fIn)
test_answers_subset = data

print(type(test_answers_subset))

<class 'list'>


In [10]:
queries = []

collection_filepath = os.path.join(data_folder, 'str_queries.json')
with open(collection_filepath, 'r', encoding='utf8') as fIn:
  data = json.load(fIn)
queries = data

print(type(queries))

<class 'list'>


In [11]:
queries_answers = []

collection_filepath = os.path.join(data_folder, 'str_queries_answers.json')
with open(collection_filepath, 'r', encoding='utf8') as fIn:
  data = json.load(fIn)
queries_answers = data

print(type(queries_answers))

<class 'list'>


In [12]:
print(len(queries))

125


In [13]:
queries[:10]

['How to convert HTML <table> to a 2D array',
 'Alternating a int array efficiently',
 'Avoid repetition and print unique ordered sets',
 'Java Runtime.exec() to run a java program',
 'Android: Time stored in db is not the same as on phone',
 'How to sort a Vector of Vectors?',
 'Java - Store data from file into Arrays',
 'Arrays.asList().contains() giving wrong result',
 'Loop through random letters without repeats',
 'Java Math.random not random in while loop']

In [14]:
cnt = 0
for idx, q in enumerate(test_corpus_subset):
    if q in queries:
        print('there a test query in corpus')
        break
    else:
        cnt += 1
print(f'finish search {cnt} queries')

finish search 7444 queries


In [15]:
print(len(test_corpus_subset))

7444


In [16]:
for set in (test_corpus_subset, queries):
    cnt = 0
    for idx, q in enumerate(set):
        if '\n' in q:
            print(f'There \n in {idx}')
        else:
            cnt += 1
    print(f'finish search {cnt} queries')

finish search 7444 queries
finish search 125 queries


In [17]:
passages = test_corpus_subset

corpus_embeddings = bi_encoder.encode(passages, convert_to_tensor=True, show_progress_bar=True)

Batches:   0%|          | 0/233 [00:00<?, ?it/s]

In [18]:
print(test_answers_subset)

["['java.lang.String.equals']", "['java.lang.String.split']", "['java.lang.String.split']", "['java.util.Arrays.sort']", "['java.lang.Object.equals']", "['java.lang.Integer.parseInt']", "['java.util.Collections.synchronizedList']", "['java.lang.String.split']", "['java.util.Collections.sort']", "['java.lang.String.equals']", "['java.util.Collections.sort']", "['java.lang.Integer.parseInt']", "['java.lang.String.split']", "['java.util.Arrays.asList']", "['java.lang.System.exit']", "['java.lang.String.split']", "['java.lang.String.matches']", "['java.lang.Thread.sleep']", "['java.lang.System.nanoTime']", "['java.lang.Object.equals']", "['java.lang.System.exit']", "['java.lang.Thread.sleep']", "['java.lang.Integer.parseInt']", "['java.lang.System.currentTimeMillis']", "['java.util.Collections.sort']", "['java.lang.String.getBytes']", "['java.lang.String.format']", "['java.lang.Thread.sleep']", "['java.lang.System.currentTimeMillis']", "['java.lang.Integer.parseInt']", "['java.lang.Double.

In [19]:
print(f'{queries[0]}---------{queries_answers[0]}')

How to convert HTML <table> to a 2D array---------['java.lang.String.split']


In [20]:
#This function will search all wikipedia articles for passages that
#answer the query
def evaluate(query,answer):


  top_k=5
  answer=eval(answer)
  print(query, answer)

#   bm25_scores = bm25.get_scores(bm25_tokenizer(query))
#   top_n = np.argpartition(bm25_scores, -50)[-50:]
#   bm25_hits = [{'corpus_id': idx, 'score': bm25_scores[idx]} for idx in top_n]
#   bm25_hits = sorted(bm25_hits, key=lambda x: x['score'], reverse=True)

#   BM25_counter = -1
#   BM25_tmp_map = 0
#   BM25_tmp_mrr = 0
#   temp_hits = 0
#   tmep_answer = answer[:]
#   for idx, hit in enumerate(bm25_hits[0:50]):
#       candidate = eval(evaluate_answers[hit['corpus_id']].replace("\n", " "))
#       for c in candidate:
#         if c in tmep_answer:
#           if BM25_counter == -1: BM25_counter = idx + 1
#           temp_hits+=1

#           BM25_tmp_map += temp_hits/(idx+1)
#           tmep_answer.remove(c)

  # print(temp_hits)
  # print(BM25_tmp_map)
#   BM25_tmp_map /= len(answer)
#   BM25_tmp_mrr = 0.0
#   if BM25_counter!= -1:
#     BM25_tmp_mrr = 1/BM25_counter



  question_embedding = bi_encoder.encode(query, convert_to_tensor=True)
  Encoder_hits = util.semantic_search(question_embedding, corpus_embeddings, top_k=top_k)
  Encoder_hits = Encoder_hits[0]  # Get the hits for the first query

  cross_inp = [[query, passages[hit['corpus_id']]] for hit in Encoder_hits]
  cross_scores = cross_encoder.predict(cross_inp)
  for idx in range(len(cross_scores)):
      Encoder_hits[idx]['cross-score'] = cross_scores[idx]


  Bi_Encoder_counter = -1
  Bi_Encoder_tmp_map = 0
  Bi_Encoder_tmp_mrr = 0

  Bi_Encoder_hit_list=[0]*top_k
  Bi_Encoder_hit_recall_list=[0]*top_k

  temp_hits = 0
  Encoder_hits = sorted(Encoder_hits, key=lambda x: x['score'], reverse=True)
  tmep_answer = answer[:]
  for idx,hit in enumerate(Encoder_hits[0:top_k]):
      candidate= eval(test_answers_subset[hit['corpus_id']].replace("\n", " "))
      for c in candidate:

        if c in answer:
          if not query==passages[hit['corpus_id']].replace("\n", " ").replace("?",""):
            Bi_Encoder_hit_list[idx]=1

        if c in tmep_answer:
          if not query==passages[hit['corpus_id']].replace("\n", " ").replace("?",""):


            if Bi_Encoder_counter == -1:
              Bi_Encoder_counter = idx + 1


              print("\t{:.3f}\t{}\t{}".format(hit['score'], passages[hit['corpus_id']].replace("\n", " "), test_answers_subset[hit['corpus_id']].replace("\n", " ")))
            temp_hits+=1


            Bi_Encoder_hit_recall_list[idx]=1


            Bi_Encoder_tmp_map += temp_hits/(idx+1)
            tmep_answer.remove(c)
  Bi_Encoder_tmp_map /= len(answer)
  Bi_Encoder_tmp_mrr = 0.0
  if Bi_Encoder_counter!= -1:
    Bi_Encoder_tmp_mrr = 1/Bi_Encoder_counter

  # answer_api= Answers[hit['corpus_id']]
  Cross_Encoder_counter = -1
  Cross_Encoder_tmp_map = 0
  Cross_Encoder_tmp_mrr = 0

  Cross_Encoder_hit_list=[0]*top_k
  Cross_Encoder_hit_recall_list=[0]*top_k

  temp_hits = 0
  Encoder_hits = sorted(Encoder_hits, key=lambda x: x['cross-score'], reverse=True)
  tmep_answer = answer[:]
  for idx,hit in enumerate(Encoder_hits[0:50]):
      # print("\t{:.3f}\t{}\t{}".format(hit['cross-score'], passages[hit['corpus_id']].replace("\n", " "), Answers[hit['corpus_id']].replace("\n", " ")))
      candidate= eval(test_answers_subset[hit['corpus_id']].replace("\n", " "))
      # print(candidate,answer)
      for c in candidate:

        if c in answer:
          if not query==passages[hit['corpus_id']].replace("\n", " ").replace("?",""):
            Cross_Encoder_hit_list[idx]=1

        if c in tmep_answer:

          if Cross_Encoder_counter == -1: Cross_Encoder_counter = idx + 1
          temp_hits+=1


          Cross_Encoder_hit_recall_list[idx]=1


          Cross_Encoder_tmp_map += temp_hits/(idx+1)
          tmep_answer.remove(c)
  Cross_Encoder_tmp_map /= len(answer)
  Cross_Encoder_tmp_mrr = 0.0
  if Cross_Encoder_counter!= -1:
    Cross_Encoder_tmp_mrr = 1/Cross_Encoder_counter



  return Bi_Encoder_counter,Bi_Encoder_tmp_mrr,Bi_Encoder_tmp_map,Cross_Encoder_counter,Cross_Encoder_tmp_mrr,Cross_Encoder_tmp_map,str(answer),Bi_Encoder_hit_list,Bi_Encoder_hit_recall_list,Cross_Encoder_hit_list,Cross_Encoder_hit_recall_list

In [21]:
# BM25_mrr = 0
# BM25_map = 0

Bi_Encoder_mrr = 0
Bi_Encoder_map = 0

Cross_Encoder_mrr = 0
Cross_Encoder_map = 0

api_list =[]
good_result=[]


Bi_Encoder_precision=[0]*4
Bi_Encoder_recall=[0]*4

Cross_Encoder_precision=[0]*4
Cross_Encoder_recall=[0]*4

for idx in range(len(queries)):
# for idx in range(10):

  Bi_Encoder_counter,Bi_Encoder_tmp_mrr,Bi_Encoder_tmp_map,Cross_Encoder_counter,Cross_Encoder_tmp_mrr,Cross_Encoder_tmp_map,answer_api,Bi_Encoder_hit_list,Bi_Encoder_hit_recall_list,Cross_Encoder_hit_list,Cross_Encoder_hit_recall_list = evaluate(query = queries[idx], answer = queries_answers[idx])

  print(Bi_Encoder_counter, Cross_Encoder_counter)

  len_api = len(eval(answer_api))
  print()

  temp_precision=[0]*4
  temp_recall=[0]*4
  for idx, n in enumerate([1,3,5,10]):
    temp_precision[idx] = sum(Bi_Encoder_hit_list[:n])/n
    temp_recall[idx] = sum(Bi_Encoder_hit_recall_list[:n])/(len_api)

  Bi_Encoder_precision = [x + y for (x, y) in zip(Bi_Encoder_precision, temp_precision)]
  Bi_Encoder_recall = [x + y for (x, y) in zip(Bi_Encoder_recall, temp_recall)]


  temp_precision=[0]*4
  temp_recall=[0]*4
  for idx, n in enumerate([1,3,5,10]):
    temp_precision[idx] = sum(Cross_Encoder_hit_list[:n])/n
    temp_recall[idx] = sum(Cross_Encoder_hit_recall_list[:n])/(len_api)

  Cross_Encoder_precision = [x + y for (x, y) in zip(Cross_Encoder_precision, temp_precision)]
  Cross_Encoder_recall = [x + y for (x, y) in zip(Cross_Encoder_recall, temp_recall)]


  # print(BM25_tmp_mrr, Bi_Encoder_tmp_mrr, Cross_Encoder_tmp_mrr)
  # print(BM25_tmp_map, Bi_Encoder_tmp_map, Cross_Encoder_tmp_map)
  # if -1<Bi_Encoder_counter < 3 or -1<Cross_Encoder_counter < 3:
  #   good_result.append([queries[idx], queries_answers[idx]])

  api_list.append(answer_api)
#   BM25_mrr+=BM25_tmp_mrr
#   BM25_map+=BM25_tmp_map

  Bi_Encoder_mrr+=Bi_Encoder_tmp_mrr
  Bi_Encoder_map+=Bi_Encoder_tmp_map

  Cross_Encoder_mrr+=Cross_Encoder_tmp_mrr
  Cross_Encoder_map+=Cross_Encoder_tmp_map

# BM25_mrr/=len(queries)
# BM25_map/=len(queries)

Bi_Encoder_mrr/=len(queries)
Bi_Encoder_map/=len(queries)

Cross_Encoder_mrr/=len(queries)
Cross_Encoder_map/=len(queries)

Bi_Encoder_precision = [x/len(queries) for x in Bi_Encoder_precision]
Bi_Encoder_recall = [x/len(queries) for x in Bi_Encoder_recall]


Cross_Encoder_precision = [x/len(queries) for x in Cross_Encoder_precision]
Cross_Encoder_recall = [x/len(queries) for x in Cross_Encoder_recall]

print("Bi_Encoder_precision")
print(Bi_Encoder_precision)

print("Bi_Encoder_recall")
print(Bi_Encoder_recall)

print("Cross_Encoder_precision")
print(Cross_Encoder_precision)

print("Cross_Encoder_recall")
print(Cross_Encoder_recall)

print(Bi_Encoder_mrr,Cross_Encoder_mrr)
print(Bi_Encoder_map,Cross_Encoder_map)
print(api_list)

How to convert HTML <table> to a 2D array ['java.lang.String.split']


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

-1 -1

Alternating a int array efficiently ['java.lang.System.arraycopy']


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

-1 -1

set of string permutation problem ['java.lang.String.split']


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

	0.969	Java String index out of bound	['java.lang.String.split']
4 2

Bi_Encoder_precision
[0.384, 0.37866666666666676, 0.368, 0.184]
Bi_Encoder_recall
[0.372, 0.556, 0.636, 0.636]
Cross_Encoder_precision
[0.44, 0.3706666666666667, 0.368, 0.184]
Cross_Encoder_recall
[0.428, 0.524, 0.632, 0.632]
0.4792000000000001 0.5066666666666668
0.4752000000000001 0.5046666666666668
["['java.lang.String.split']", "['java.lang.System.arraycopy']", "['java.lang.String.split']", "['java.lang.Runtime.exec']", "['java.lang.System.currentTimeMillis']", "['java.util.Collections.sort']", "['java.lang.String.split']", "['java.util.Arrays.asList']", "['java.util.Collections.shuffle']", "['java.lang.Math.random']", "['java.lang.Double.parseDouble']", "['java.lang.String.split']", "['java.util.Arrays.asList']", "['java.lang.String.split']", "['java.lang.System.currentTimeMillis']", "['java.lang.System.arraycopy']", "['java.lang.String.replaceAll']", "['java.lang.Integer.parseInt']", "['java.lang.Double.parseDou